In [131]:
import sys
try:
    from pyathena import connect
except:
    print("Failed to import pyathena, trying to install it")
    !{sys.executable} -m pip install PyAthena
    from pyathena import connect
    print("successfully installed")
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
import pandas as pd
import boto3
import boto3
import base64
from botocore.exceptions import ClientError
from datetime  import datetime, timedelta, date
import time

In [132]:
import creds

In [1]:
# if using jupyterlab and plotly doesn't work https://plot.ly/python/getting-started/#jupyterlab-support-python-35
install_jupyterlab_plotly_extensions_command = """
# Avoid "JavaScript heap out of memory" errors during extension installation
# (OS X/Linux)
export NODE_OPTIONS=--max-old-space-size=4096
# (Windows)
set NODE_OPTIONS=--max-old-space-size=4096

# Jupyter widgets extension
jupyter labextension install @jupyter-widgets/jupyterlab-manager@1.0 --no-build

# jupyterlab renderer support
jupyter labextension install jupyterlab-plotly@1.1.0 --no-build

# FigureWidget support
jupyter labextension install plotlywidget@1.1.0 --no-build

# JupyterLab chart editor support (optional)
jupyter labextension install jupyterlab-chart-editor@1.2 --no-build

# Build extensions (must be done to activate extensions since --no-build is used above)
jupyter lab build

# Unset NODE_OPTIONS environment variable
# (OS X/Linux)
unset NODE_OPTIONS"""



    

In [7]:
import plotly.graph_objects as go
import plotly as py

import plotly.offline as off
import plotly.graph_objs as go
import numpy as np
from plotly.subplots import make_subplots

#off.init_notebook_mode(connected=True)

In [8]:

fig = make_subplots(1, 1)

fig.add_trace(
    go.Pie(values=[1,2,3], labels=["a", "b", "c"]),
)

                
fig.show()

In [135]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3') #slower, but easier
# s3_conn = boto3.s3.connect_to_region('us-west-2')

athena = boto3.client('athena')

# Settings

In [147]:
#--- Analysis Settings --
num_days_to_analyse = 8
analysis_end_time = datetime.fromordinal(date.today().toordinal()) # start of today as we only get yesterday's data.
analysis_start_time = analysis_end_time - timedelta(days=num_days_to_analyse)
country_code_to_analyse = "ID"


#--- Core Settings -- 
event_log_s3_path = "s3://ms-data-pipeline-production/ms-data-stream-production-processed"
event_log_s3_bucket = event_log_s3_path.split("s3://")[1].split("/")[0]
event_log_s3_prefix = event_log_s3_path.split("/")[-1]
athena_bucket_path = "s3://aws-athena-query-results-358002497134-ap-southeast-1/"
athena_database = "ms_data_processed_production"
athena_year_partition = "partition_0" #2019
athena_month_partition = "partition_1" #02
athena_day_partition = "partition_2" #02
athena_raw_events_table = "ms_data_stream_production_processed"
athena_easy_events_table =  "id_ab_test"
aws_region_name = "ap-southeast-1"


# --
list_out_s3_bucket = False #extremely slow.


# Getting Base Data

## Events from Athena

In [148]:
print("Connecting to Athena")
athena_conn = connect(s3_staging_dir=athena_bucket_path, region_name=aws_region_name)

Connecting to Athena


In [149]:
print("Running small test query to check the athena connection")
pd.read_sql("select * from "+athena_database+"."+athena_raw_events_table+" limit 10", athena_conn)

Running small test query to check the athena connection


,type,context,user,body,sent_at,partition_0,partition_1,partition_2
0,page,{title=Travel Insurance 2019 - Singapore | Mon...,{anonymous_id=7f7eabc8-fe51-429e-8e0a-e2eb983d...,"{event_name=PageView, data={title=null, status...",2019-09-22T18:53:17.878Z,2019,09,23
1,event,{title=Beli Rumah Lelang Sitaan Bank Pasti Leb...,"{anonymous_id=null, amp_id=amp-m45NpgpPYK6ybrd...","{event_name=Reading, data={title=null, status=...",2019-09-22T18:53:19.549000+00:00,2019,09,23
2,event,{title=Kerja Malam Dilakoni para Pekerja denga...,{anonymous_id=ff969c33-b3b6-4b58-a16e-05b23bbd...,"{event_name=Reading, data={title=Kerja Malam D...",2019-09-22T18:53:18.516Z,2019,09,23
3,event,{title=Mengintip Rumah Mayangsari yang Megah d...,{anonymous_id=d8773ea0-3dc9-40e3-beb9-8296967d...,"{event_name=Reading, data={title=Mengintip Rum...",2019-09-22T18:53:20.775Z,2019,09,23
4,event,"{title=Rumah Iis Dahlia Harganya Rp 17 Miliar,...",{anonymous_id=01a23afd-f512-4db2-bb68-687c6a55...,"{event_name=Reading, data={title=Rumah Iis Dah...",2019-09-22T18:53:20.018Z,2019,09,23
5,page,"{title=全港最佳信用卡 2019 | MoneySmart.hk, page_url=...",{anonymous_id=8a7ab3a1-3746-4b35-8bbe-454b17c6...,"{event_name=PageView, data={title=null, status...",2019-09-22T18:53:19.939Z,2019,09,23
6,page,{title=Muzdalifah Punya 6 Sudut Rumah Megah ya...,{anonymous_id=9cfab873-3c35-49ba-9952-3c231cc6...,"{event_name=PageView, data={title=null, status...",2019-09-22T18:53:22.852Z,2019,09,23
7,event,{title=5 Resep Jajanan Kekinian Bujet Rp 30 Ri...,"{anonymous_id=null, amp_id=w5MJ260GaGR0KCZpC21...","{event_name=Reading, data={title=null, status=...",2019-09-22T17:53:42.578000+00:00,2019,09,23
8,event,{title=Bambang Irianto Ditetapkan Tersangka KP...,{anonymous_id=dea6b68f-acb4-42a2-8be0-041abf9b...,"{event_name=Reading, data={title=Bambang Irian...",2019-09-22T18:53:20.441Z,2019,09,23
9,page,{title=Mau Beli Mobil Antik Mercy Tiger? Siapi...,"{anonymous_id=null, amp_id=s-LZgJ7zh1lc8j5ey_L...","{event_name=PageView, data={title=null, status...",2019-09-22T18:53:20.270000+00:00,2019,09,23


In [150]:
def create_blog_pageview_and_scroll_query(country_code, from_datetime, to_datetime, limit=None, include_device_type_data = True):
    """ 
    This has been taken from the view in athena to make it more flexible.
    country_code should be e.g. TW / SG / ...
    
    It filters out just events from the blog
    """
    print("Getting the pageview and read events from %s to %s"%(from_datetime.isoformat(), to_datetime.isoformat()))
    
    country_code_to_url_core = {
        "ID":"moneysmart.id", 
        "PH":"moneysmart.ph", 
        "TW":"moneysmart.tw", 
        "SG":"blog.moneysmart.sg", 
        "HK":"blog.moneysmart.hk"
        
    }
    
    if country_code not in country_code_to_url_core:
        raise Exception("Didn't recognise country code"+repr(country_code)+".")
        
    #These are just used for the partition selection and further filter is done on the datetime level.
    start_year_inclusive = from_datetime.year
    end_year_inclusive = to_datetime.year
    start_month_inclusive = from_datetime.month
    end_month_inclusive = to_datetime.month
    start_day_inclusive = from_datetime.day
    end_day_inclusive = to_datetime.day
        
    query = """
    SELECT
      CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , (CASE "body"."data"."status" 
        WHEN 'Page Bottom Reached' THEN 100 
        WHEN 'Article Reading Completed' THEN 100 WHEN 'Article Body 100' THEN 100 
        WHEN 'Article Reading 75%' THEN 75 WHEN 'Article Body 75' THEN 75 
        WHEN 'Article Reading 50%' THEN 50 WHEN 'Article Body 50' THEN 50 
        WHEN 'Article Reading 25%' THEN 25 WHEN 'Article Body 25' THEN 25
        WHEN 'Article Reading Started' THEN 0 WHEN 'Article Loaded' THEN 0 
        ELSE 0 END) "article_read_depth"
    , "user"."anonymous_id"
    , "split_part"("context"."page_url", '?', 1) "canonical_url"
    , "context"."page_url"
    , "context"."referrer"
    , CAST("strpos"("context"."page_url", '?amp') AS boolean) "is_amp"
    , CAST((("context"."page_url" LIKE '%utm_medium%') OR ("context"."page_url" LIKE '%gclid%')) AS boolean) "has_marketing_param"
    , CAST("strpos"("context"."page_url", '://www3.') AS boolean) "is_test"
    , CAST("strpos"("context"."page_url", '://www.') AS boolean) "is_control"
    """
    if include_device_type_data:
        query += """
        , context.device.type as device_type
        , context.browser.name as browser_name
        , context.browser.major as browser_major_version
        , context.browser.version as browser_version
        
        , context.operating_system.name as operating_system_name
        , context.operating_system.version as operating_system_version
        , context.user_agent as user_agent
        """
        
    query += """
    FROM
      """+athena_database+ "." +athena_raw_events_table+ """
    WHERE 
        (("body"."event_name" = 'Reading') OR ("body"."event_name" = 'PageView')) 
        
"""
    
    country_constraint = """ AND 
            ("regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) 
            LIKE '%{}%')""".format(country_code_to_url_core[country_code])

    
    partition_constraints = "\n AND ("
    partition_constraints += "\n " + athena_year_partition + " >= '%.4i'"%start_year_inclusive
    partition_constraints += "\n AND " + athena_month_partition + " >= '%.2i'"%start_month_inclusive
    partition_constraints += "\n AND " + athena_day_partition+ " >= '%.2i'"%start_day_inclusive
    
    #TOD: >> I think this needs some adjustment for when you roll between months.
    partition_constraints += "\n AND " + athena_year_partition + " <= '%.4i'"%end_year_inclusive
    partition_constraints += "\n AND " + athena_month_partition + " <= '%.2i'"%end_month_inclusive
    partition_constraints += "\n AND " + athena_day_partition+ " <= '%.2i'"%end_day_inclusive
    partition_constraints += "\n) "
     # TODO: + datetime further constraints
    
    query = query + " "+country_constraint
    query = query+ " " + partition_constraints
    
    if limit:
        query += " LIMIT %i"%limit
    
    return query
    
def get_blog_events(country_code, from_datetime, to_datetime, limit=None):
    #TODO: ideally you break into chunks during reading and replace various strings with post_id and set data types to more efficient ones (category)
    start_time =  datetime.now()
    print("Starting athena events query at %s"% start_time.isoformat())
    print("This might take some time")
    
    query = create_blog_pageview_and_scroll_query(country_code, from_datetime, to_datetime, limit=limit)
    
    print("Query : "+ query)
    
    events = pd.read_sql(query, athena_conn)
    
    end_time =  datetime.now()
    print("Ended query at %s"%end_time.isoformat())
    dt = end_time-start_time
    print("Query took %s" %dt)
    return events
    

In [151]:
events = get_blog_events(country_code_to_analyse, analysis_start_time, analysis_end_time, limit =None)


Starting athena events query at 2019-10-21T09:12:51.922615
This might take some time
Getting the pageview and read events from 2019-10-13T00:00:00 to 2019-10-21T00:00:00
Query : 
    SELECT
      CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , (CASE "body"."data"."status" 
        WHEN 'Page Bottom Reached' THEN 100 
        WHEN 'Article Reading Completed' THEN 100 WHEN 'Article Body 100' THEN 100 
        WHEN 'Article Reading 75%' THEN 75 WHEN 'Article Body 75' THEN 75 
        WHEN 'Article Reading 50%' THEN 50 WHEN 'Article Body 50' THEN 50 
        WHEN 'Article Reading 25%' THEN 25 WHEN 'Article Body 25' THEN 25
        WHEN 'Article Reading Started' THEN 0 WHEN 'Article Loaded' THEN 0 
        ELSE 0 END) "article_read_depth"
    , "user"."anonymous_id"
    , "split_part"("context"."page_url", '?', 1) "canonical_url"
    , "context"."page_url"
    , "context"."re

In [152]:
print("Got %i events"%len(events))
events.head(5)

Got 836324 events


,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-10-12 18:52:28.633,2019-10-12T18:52:28.633Z,page,PageView,None,0,4a298a33-84b7-444e-96d7-37fe20889347,https://www.moneysmart.id/mobil-arteria-dahlan/,https://www.moneysmart.id/mobil-arteria-dahlan...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,7.1.1,Mozilla/5.0 (Linux; Android 7.1.1; CPH1717 Bui...
1,2019-10-12 18:52:29.893,2019-10-12T18:52:29.893Z,page,PageView,None,0,29739869-94bd-4c21-8d89-751ef2b204b6,https://www.moneysmart.id/keluarga-halilintar-...,https://www.moneysmart.id/keluarga-halilintar-...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/11.5.1 (Linux; U; Android 27; e...
2,2019-10-12 18:52:29.160,2019-10-12T18:52:29.160Z,page,PageView,None,0,a954d9ba-b4ee-47d2-b09e-cd0ace077603,https://www.moneysmart.id/tahir-masuk-jajajara...,https://www.moneysmart.id/tahir-masuk-jajajara...,,...,True,False,True,mobile,None,None,None,Android,28,BacaBerita App/12.2.2 (Linux; U; Android 28; e...
3,2019-10-12 18:52:30.135,2019-10-12T18:52:30.135Z,event,Reading,Article Body 50,50,f25077c7-cfbc-49ab-a6c9-262a2376be45,https://www3.moneysmart.id/keluarga-halilintar...,https://www3.moneysmart.id/keluarga-halilintar...,android-app://com.google.android.googlequickse...,...,False,True,False,mobile,Chrome,77,77.0.3865.73,Android,9,Mozilla/5.0 (Linux; Android 9; SM-A730F) Apple...
4,2019-10-12 18:52:29.005,2019-10-12T18:52:29.005Z,event,Reading,Article Body 100,100,94937adc-1315-4484-a47d-43895385beda,https://www.moneysmart.id/presiden-ri-dan-peso...,https://www.moneysmart.id/presiden-ri-dan-peso...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,7.1.2,Mozilla/5.0 (Linux; Android 7.1.2; Redmi Note ...


In [153]:
pageview_events = events[events.event_name=="PageView"]

In [154]:
pageview_events.head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-10-12 18:52:28.633,2019-10-12T18:52:28.633Z,page,PageView,None,0,4a298a33-84b7-444e-96d7-37fe20889347,https://www.moneysmart.id/mobil-arteria-dahlan/,https://www.moneysmart.id/mobil-arteria-dahlan...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,7.1.1,Mozilla/5.0 (Linux; Android 7.1.1; CPH1717 Bui...
1,2019-10-12 18:52:29.893,2019-10-12T18:52:29.893Z,page,PageView,None,0,29739869-94bd-4c21-8d89-751ef2b204b6,https://www.moneysmart.id/keluarga-halilintar-...,https://www.moneysmart.id/keluarga-halilintar-...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/11.5.1 (Linux; U; Android 27; e...
2,2019-10-12 18:52:29.160,2019-10-12T18:52:29.160Z,page,PageView,None,0,a954d9ba-b4ee-47d2-b09e-cd0ace077603,https://www.moneysmart.id/tahir-masuk-jajajara...,https://www.moneysmart.id/tahir-masuk-jajajara...,,...,True,False,True,mobile,None,None,None,Android,28,BacaBerita App/12.2.2 (Linux; U; Android 28; e...
5,2019-10-12 18:52:27.833,2019-10-12T18:52:27.833Z,page,PageView,None,0,6e600b3b-a905-4281-b9cb-8a9c595276bf,https://www.moneysmart.id/film-live-action-dis...,https://www.moneysmart.id/film-live-action-dis...,https://www.google.co.id/search?q=film+powerfu...,...,False,False,True,mobile,UCBrowser,12,12.13.0.1207,Android,8.1.0,Mozilla/5.0 (Linux; U; Android 8.1.0; en-US; v...
7,2019-10-12 18:52:30.706,2019-10-12T18:52:30.706Z,page,PageView,None,0,92395e5b-2486-4f9a-bac6-c6b7e9371068,https://www.moneysmart.id/keluarga-halilintar-...,https://www.moneysmart.id/keluarga-halilintar-...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/12.1.4 (Linux; U; Android 27; e...


In [106]:
events.head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,is_amp,has_marketing_param,is_test,is_control
0,2019-09-19 19:44:10.463,2019-09-19T19:44:10.463Z,page,PageView,None,0,1ca9fcd7-16c8-4d12-85bb-da3ab4c0ffc4,https://www.moneysmart.id/koleksi-kendaraan-bj...,https://www.moneysmart.id/koleksi-kendaraan-bj...,http://m.facebook.com/,False,True,False,True
1,2019-09-19 19:44:09.133,2019-09-19T19:44:09.133Z,event,Reading,Article Body 100,100,c3e6e222-70f3-4181-9c2d-f51592710a68,https://www.moneysmart.id/artis-indonesia-dulu...,https://www.moneysmart.id/artis-indonesia-dulu...,http://m.facebook.com/,False,True,False,True
2,2019-09-19 19:56:04.395,2019-09-19T19:56:04.395000+00:00,event,Reading,Article Body 50,50,None,https://www.moneysmart.id/4-cara-nonton-stream...,https://www.moneysmart.id/4-cara-nonton-stream...,,True,False,False,True
3,2019-09-19 19:56:01.573,2019-09-19T19:56:01.573Z,page,PageView,None,0,9e811884-49e2-4032-9540-0c2a54734c92,https://www.moneysmart.id/fakta-sandhyca-putri...,https://www.moneysmart.id/fakta-sandhyca-putri...,http://m.facebook.com/,False,True,False,True
4,2019-09-19 19:56:02.491,2019-09-19T19:56:02.491Z,page,PageView,None,0,4ef346ca-f0af-4dfb-b475-86d781ddce5c,https://www.moneysmart.id/saham-perusahaan-mer...,https://www.moneysmart.id/saham-perusahaan-mer...,,False,True,False,True


## Convert Raw Pageview Events Into Scroll Depth Summary

Basically re-creating:

```
CREATE OR REPLACE VIEW id_pageview_metrics AS 
SELECT
  "anonymous_id"
, "canonical_url"
, "substr"("sent_at", 1, 10) "day_str"
, "min"("sent_at_timestamp") "from_time"
, "max"("sent_at_timestamp") "to_time"
, "date_diff"('second', "min"("sent_at_timestamp"), "max"("sent_at_timestamp")) "time_on_page_seconds"
, "max"("is_test") "is_test"
, "max"("article_read_depth") "max_article_read_depth"
FROM
  id_ab_test
WHERE (NOT "is_amp")
GROUP BY 1, 2, 3   >>> should it not have is_test in the group by????
```

In [113]:
# Group by page and user to within the same day to get an idea of engagement within the page 
# (if there's only one event on the page, then it's likely there was no 2nd pageview, not perfect though.)

# TODO: add is_amp

engagement_summary = events.groupby(["anonymous_id", "canonical_url", events.sent_at_timestamp.dt.floor('d'), "is_test", "is_amp"])\
    .aggregate({"sent_at_timestamp": [np.min, np.max], "article_read_depth":np.max }, axis="columns")

engagement_summary["time_on_page_seconds"] = engagement_summary.apply(lambda row: row[] - row[], axis=1)

#>>> TODO: add the time on the page as time_on_page_seconds, which I think you could do as appending a column

#non_amp_engagement_summary = engagement_summary[engagement_summary.is_amp==False]

TypeError: unsupported operand type(s) for -: 'function' and 'function'

In [118]:
engagement_summary.reset_index(inplace=True)
engagement_summary.head()


anonymous_id  \
                                          
0  00005691-02d9-48c2-ba12-b8939ccbce34   
1  0000b293-f267-410d-85fc-5895dc68aa3b   
2  0000cc5d-4dd3-46ac-80e0-567192a868fe   
3  0000e1aa-7786-4276-b928-367a16e0901b   
4  00010b6a-aa66-4d61-8817-e4963e43a24f   

                                       canonical_url sent_at_timestamp  \
                                                                         
0  https://www.moneysmart.id/utang-luar-negeri-in...        2019-09-20   
1  https://www.moneysmart.id/rumah-luwih-hotel-ba...        2019-09-20   
2  https://www.moneysmart.id/gak-sesulit-yang-dib...        2019-09-20   
3  https://www.moneysmart.id/agnez-mo-pakai-baran...        2019-09-20   
4  https://www.moneysmart.id/johan-budi-memiliki-...        2019-09-20   

  is_test is_amp       sent_at_timestamp                          \
                                    amin                    amax   
0   False  False 2019-09-20 10:00:45.039 2019-09-20 10:01:20.870   
1   False  False 2019-09-20 00:50:23.083 2019-09-20 00:52:23.595   
2   False  False 2019-09-20 08:26:14.978 2019-09-20 08:27:21.581   
3   False  False 2019-09-20 12:52:48.356 2019-09-20 12:53:17.308   
4   False  False 2019-09-20 14:15:56.913 2019-09-20 14:18:05.452   

  article_read_depth  
                amax  
0                 50  
1                100  
2                 75  
3                100  
4                100

# Queries to Add
```

select * from id_pageview_metrics where time_on_page_seconds <5 and max_article_read_depth = 100 limit 100
```
---
```
select * from id_engagement_summary order by day_str, is_test 

```

In [ ]:


# TODO: the right way to do this would be to store connection details in the secrets manager.  Unfortunately not working :(
def get_secret():

    secret_name = "data_pier"
    region_name = "ap-southeast-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )


    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # I've deleted a chunk of error handling bumpf

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )


def get_data_pier_connection():
    #db_pass = input()
    db_pass = creds.pg_pass
    db_host = "data-pier-production.cl8qfdl47mtr.ap-southeast-1.rds.amazonaws.com"
    db_user = "jamie"
    db_database = "data_pier"
    pg_conn = psycopg2.connect(host=db_host, user = db_user, dbname=db_database, password= db_pass)
    pg_conn.set_session(readonly=True, autocommit=True) #without this it can lock tables.
    
    
    return pg_conn, pg_conn.cursor()


In [195]:
pg_conn, cursor = get_data_pier_connection()
cursor.execute("select * from information_schema.tables limit 10")

 BVHSjgnqHpqPRgHHaG9sYdJqD94ZeykJ


In [199]:
#query = "select * from information_schema.columns where table_schema='id_wordpress'" 
query = "select  * from id_wordpress.wp_terms join id_wordpress.wp_posts where slug like '%kartu%' limit 100" # gets slug
query = """
        select * from id_wordpress.wp_postmeta where meta_key not like'%oembed%' 
        and meta_key like '%slug%'
        order by meta_key desc limit 100
        """

query = "select taxonomy, count(*) from id_wordpress.wp_term_taxonomy group by taxonomy"
query = "select * from id_wordpress.wp_term_taxonomy where taxonomy='post_tag'"
pd.read_sql_query(query, pg_conn).head()

,term_taxonomy_id,term_id,taxonomy,description,parent,count
0,577.0,577.0,post_tag,,0.0,133
1,582.0,582.0,post_tag,,0.0,20
2,583.0,583.0,post_tag,,0.0,47
3,586.0,586.0,post_tag,,0.0,117
4,591.0,591.0,post_tag,,0.0,56


In [119]:
pps_query = """
select users.day_str, users.is_test, num_users, num_pageviews, cast(num_pageviews as DOUBLE) / (num_users) as pageviews_per_user_per_day from 

(select day_str, is_test, count(distinct(anonymous_id)) AS num_users from id_pageview_metrics group by day_str, is_test) as users

JOIN

(select day_str, is_test, count(*) as num_pageviews from id_pageview_metrics group by day_str, is_test) as pageviews

ON 
 users.day_str = pageviews.day_str
 and 
 users.is_test = pageviews.is_test
order by day_str, is_test
"""
pageviews_per_session = pd.read_sql_query(pps_query, athena_conn)

In [84]:
pageviews_per_session


,day_str,is_test,num_users,num_pageviews,pageviews_per_user_per_day
0,2019-08-29,False,22667,27063,1.193938
1,2019-08-29,True,778,1003,1.289203
2,2019-08-30,False,147652,178157,1.206601
3,2019-08-30,True,5432,6586,1.212445
4,2019-08-31,False,146489,177051,1.208630
5,2019-08-31,True,4999,5962,1.192639
6,2019-09-01,False,150252,180434,1.200876
7,2019-09-01,True,5429,6314,1.163013
8,2019-09-02,False,150894,182569,1.209916
9,2019-09-02,True,5893,7009,1.189377


In [85]:
pageviews_per_session.groupby(['is_test']).mean()

,num_users,num_pageviews,pageviews_per_user_per_day
is_test,,,
False,130656.857143,157664.428571,1.205112
True,4778.571429,5664.714286,1.198417


In [308]:
post_slugs_to_id_df.head()

,post_id
0,153.0
1,155.0
2,158.0
3,161.0
4,199.0


In [210]:
def get_all_categories_tags(conn, schema):
    
    """domain.com/<slug> (we can parse the slug based on URL single post format)
Get the post from slug
```sql
SELECT wp_posts.*
FROM wp_posts 
WHERE 1=1 
AND wp_posts.post_name = '<slug>'
AND wp_posts.post_type = 'post' 
ORDER BY wp_posts.post_date DESC
```
Get categories of a post (post_id is from post retrieved above). For tag, replace 'category' by 'tag'
```sql
SELECT t.term_id, tt.parent, tt.count, tt.taxonomy
FROM wp_terms AS t 
INNER JOIN wp_term_taxonomy AS tt
ON t.term_id = tt.term_id
INNER JOIN wp_term_relationships AS tr
ON tr.term_taxonomy_id = tt.term_taxonomy_id
WHERE tt.taxonomy IN ('category')
AND tr.object_id IN (<post_id>)
ORDER BY t.name ASC
```
Get details of a term (category or tag) from IDs retrieved above
```sql
SELECT t.*, tt.*
FROM wp_terms AS t
INNER JOIN wp_term_taxonomy AS tt
ON t.term_id = tt.term_id
WHERE t.term_id = <term_id>
```"""
    
    
    tags_categories_query = """
    select 
        wp_posts.id as post_id, 
        -- wp_posts.post_name as slug, 
        cats.name, 
        cats.term_id, 
        cats.parent, 
        cats.count, 
        cats.taxonomy
    
    from {schema}.wp_posts
    join (
        SELECT tr.object_id, t.term_id, tt.parent, tt.count, tt.taxonomy, t.name
        FROM {schema}.wp_terms AS t 
        INNER JOIN {schema}.wp_term_taxonomy AS tt
        ON t.term_id = tt.term_id
        INNER JOIN {schema}.wp_term_relationships AS tr
        ON tr.term_taxonomy_id = tt.term_taxonomy_id
         WHERE tt.taxonomy IN ('category', 'post_tag')
       
        
    ) as cats
    on cats.object_id = wp_posts.id
    
    where
    wp_posts.post_type = 'post'
    and wp_posts.post_status = 'publish'
    
    """.format(schema=schema)

    
    
    tags_categories = pd.read_sql_query(tags_categories_query, conn)
    #print(tags_categories[:100])
    
    print(tags_categories.head())
    tags_categories.astype({"post_id":int, "term_id":int, "parent":int, "taxonomy": "category"}) #untested type casting
    
    
    tags = tags_categories[tags_categories.taxonomy=='post_tag'][['post_id','name']]
    
    categories = tags_categories[tags_categories.taxonomy=='category'][['post_id','name']]
    
    
    return categories, tags


def get_post_data(conn, schema):
    query = """
    select id as post_id, post_name as slug, post_author as author_id, CHAR_LENGTH(post_content) as post_num_chars, post_date, post_modified 
    from {schema}.wp_posts
    
    where 
    wp_posts.post_type = 'post'
    and wp_posts.post_status = 'publish'
    """.format(schema=schema)
    df = pd.read_sql_query(query, conn)
    df.astype({"post_id":int, "author_id":int}) #untested type casting
    return df
    
def slug_to_url(slug):
    return "https://www.moneysmart.id/"+slug



post_data = get_post_data(pg_conn, "id_wordpress")
    
post_slugs_to_id_df = post_data[["post_id",]] #.index(post_data["slug"])

categories, tags = get_all_categories_tags(pg_conn, "id_wordpress")
                                           

   post_id                       name  term_id  parent  count  taxonomy
0    153.0                   Pinjaman    528.0  1180.0    504  category
1    155.0                  Budgeting   1193.0  1191.0    616  category
2    155.0                 tips hemat   1299.0     0.0    175  post_tag
3    155.0  mengatur keuangan pribadi   1944.0     0.0      5  post_tag
4    155.0              mengatur uang   3053.0     0.0      5  post_tag


In [226]:
def convert_canonical_url_to_slug(url):
    if "//" not in url:
        raise Exception("expected start of url in {}".format(url))
    id_root = "https://www.moneysmart.id"
    ms_id_index = url.find(id_root)
    if not ms_id_index:
        raise Exception("{url} doesn't look like a canonical url".format(url = url))
        
    slug = url[ms_id_index+ len(id_root)+1:].strip("/")
    
    if slug == "":
        slug ="/"
    return slug
        
def is_article_slug(slug):
    if "/" in slug or len(slug)==0:
        return False
    return True

def get_post_id_from_slug(slug):
    post_id = post_slugs_to_id_df.loc[slug, :][0]
    return post_id


def get_post_id_from_url(url):
    #Expects only post urls
    slug = convert_canonical_url_to_slug(url)
    print(slug)
    post_id = get_post_id_from_slug(slug)
    return post_id

In [307]:
get_post_id_from_url("htts://www.moneysmart.id/cara-cepat-melunasi-kpr")

cara-cepat-melunasi-kpr


KeyError: 'cara-cepat-melunasi-kpr'

In [183]:
categories.head(5)

,id,slug,name
0,153.0,cara-cepat-melunasi-kpr,Pinjaman
1,155.0,mengatur-keuangan-pribadi,Budgeting
6,158.0,kpr-rumah-dan-cara-memilihnya,Pinjaman
10,161.0,agar-otak-selalu-segar,Ragam
11,171.0,tips-menambah-teman-dan-berbelanja-saat-berlibur,Kuliner &amp; Travel


In [152]:
article_categories = categories.groupby('slug').aggregate(lambda x:x.unique().tolist())
article_tags = tags.groupby('slug').aggregate(lambda x:x.unique().tolist())

In [141]:
#[z for z in article_categories.values if len(z[1])>0]
article_categories
article_categories['num_categories'] = article_categories['name'].apply(len)

In [177]:
#TODO: sometime should do all this based on post_id

def get_categories_tags_for_url(slug):
    #cats = categories[categories.slug==slug]
    try:
        tags = article_tags.loc[slug, :][1]
    except:
        print("no tags")
        tags = []
    try:
        cats = article_categories.loc[slug, :][1]
    except:
        print("no categories for %s"%slug)
        cats = []
    return cats, tags

def get_category_tags_for_url(slug):
    """
    this returns just the first category (and maybe not even the primary category... yet)
    It's useful for grouping easily by category without duplication.
    """
    cats, tags = get_categories_tags_for_slug()

start_time = time.time()
cats, tags = get_categories_tags_for_url("pembalap-indonesia-berkarier-di-mancanegara")
end_time = time.time()

print("it took ")
print(end_time-start_time)
cats_tags

it took 
0.0007693767547607422


(['Inspirasi'],
 ['pembalap',
  'pembalap indonesia',
  'pembalap indonesia berkarier hingga mancanegara'])

In [306]:
#pageviews_with_categories = pageview_events.join()
events.head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,is_amp,has_marketing_param,is_test,is_control
0,2019-09-03 23:47:24.676,2019-09-03T23:47:24.676Z,event,Reading,Article Body 50,50,23e29496-49af-4b8c-b7a3-6755efb15a22,https://www.moneysmart.id/usaha-modal-10-juta-...,https://www.moneysmart.id/usaha-modal-10-juta-...,False,True,False,True
1,2019-09-03 23:47:23.679,2019-09-03T23:47:23.679Z,event,Reading,Article Body 50,50,ea8cb9ae-bf5f-4f4d-9bf8-a0b912c0c3cf,https://www.moneysmart.id/ustadz-abdul-somad-m...,https://www.moneysmart.id/ustadz-abdul-somad-m...,False,True,False,True
2,2019-09-03 23:47:22.269,2019-09-03T23:47:22.269Z,page,PageView,None,0,5892e337-2770-4314-8eea-3f5a76c721be,https://www.moneysmart.id/keluarga-terkaya-di-...,https://www.moneysmart.id/keluarga-terkaya-di-...,False,True,False,True
3,2019-09-03 23:47:22.059,2019-09-03T23:47:22.059Z,event,Reading,Article Body 50,50,5a10018c-1585-498f-a5cd-3a7c79cfbb6f,https://www.moneysmart.id/rumah-singgah-pasha-...,https://www.moneysmart.id/rumah-singgah-pasha-...,False,True,False,True
4,2019-09-03 23:47:22.134,2019-09-03T23:47:22.134Z,event,Reading,Article Body 25,25,9d0bea57-cb28-4720-8b81-ae529b15ad58,https://www.moneysmart.id/panji-trihatmodjo-ge...,https://www.moneysmart.id/panji-trihatmodjo-ge...,False,True,False,True


### Scalyr Setup

Functions to use later.  See https://www.scalyr.com/help/api#query

In [ ]:
def query_scalyr():
    api_key = creds.scalyr_api_key
    query_url = https://www.scalyr.com/api/query
    log_file_query_id_prod_pwa = "log=*fe-pwa-id-production*"
    log_file_query_id_pod_wordpress = 
    ok_statuses = list(range(200,210))+ list(range(300, 310))
    meh_statuses = [404,]
    bad_statuses = list(range(500, 510))
    print("Bad statuses", bad_statuses)
    
"""
{
  "token":             "xxx",
  "queryType":         "log",
  "filter":            "...",
  "startTime":         "...",
  "endTime":           "...",
  "maxCount":          nnn,
  "pageMode":          "...",
  "columns":           "...",
  "continuationToken": "...",
  "priority":          "..."
}
"""

"""
{
  "status": "success",
  "matches": [
    {
      "timestamp": "1393009097459537089",
      "message": "hello, world",
      "severity": 3,
      "session": "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd",
      "thread": "28",
      "fields": {
        "tag": "foo"
      }
    }
  ],
  "sessions": {
    "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd": {
      "serverHost": "some.host.name",
      "serverIP": "1.2.3.4",
      "session": "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd"
    }
  },
  "executionTime": 12,
  "continuationToken": "L6_ckjI4BgATVPZkxGaUwQ--"
}

"""


# Data Cleaning

* Remove what looks like bots / any really heavy users (not sure if most bots will trigger analytics events)


In [13]:
non_amp_events = events[events.is_amp==False]

# Sense Checks

## Overall AB Split

Theories about why the AB split won't match what is "set" on the AB splitter:

* Repeat users will almost always be in the control set, and we have built up a good backlog over a couple of months (40% of our users are returning from google analytics, but that is over full lifetime of the cookie https://analytics.google.com/analytics/web/?authuser=2#/report/visitors-type/a117250120w173910722p173182279/ )
* Potenially it's related to coming from AMP (don't think so)
* Potentially it's based on origin of the traffic (e.g. certain query params, but this has been turned off).

In [3]:
ab_split_summary = non_amp_events.groupby("is_test").count()["sent_at"]
ab_split_summary_pretty = [[("Test (v2)" if z[0] else "Control (v1)"), z[1]] for z in ab_split_summary.iteritems()]


import plotly.graph_objects as go
fig = make_subplots(1, 1)

fig.add_trace(
    go.Pie(values=[z[1] for z in ab_split_summary_pretty], labels=[z[0] for z in ab_split_summary_pretty]),
)

                
fig.show()

NameError: name 'non_amp_events' is not defined

## Type of Event

In [266]:
event_name_counts = events.groupby(events.event_name)
event_name_counts


In [305]:
# Doing this sense check on the event fields, not the computed depth
expected_statuses = ["Article Body {}".format(i) for i in [0, 25, 50, 75, 100]]
expected_statuses_set = set(expected_statuses)
reading_status_counts = events[events.event_name=="Reading"].groupby([events.status, events.is_test]).count()['sent_at'] #[["event_name", "is_test" ], :]
reading_status_counts.sort_index()
reading_status_counts = [[z[0][0], z[0][1], z[1]] for z in reading_status_counts.iteritems()]
test_reading_status_counts = [z for z in reading_status_counts if z[1]==True]
control_reading_status_counts = [z for z in reading_status_counts if z[1]==True]

for name, reading_statuses in [["Test", test_reading_status_counts], ["Control", control_reading_status_counts]]:
    statuses_set = set([z[0] for z in reading_statuses])
    missing_statuses = list(expected_statuses_set - statuses_set)
    extra_statuses = list(statuses_set - expected_statuses_set)
    
    if len(missing_statuses)>0:
        print("ERROR: Missing expected statuses for Reading event in "+name+": " + ", ".join(missing_statuses))
        
    if len(extra_statuses)>0:
        print("ERROR: Unexpected statuses for Reading event in "+name+": " + ", ".join(extra_statuses))
                        
# Going to look at scroll depth comparison lower down, so don't need to worry about it just now.


ERROR: Missing expected statuses for Reading event in Test: Article Body 0
ERROR: Missing expected statuses for Reading event in Control: Article Body 0


## Pageviews Grouped

* By category
* By device
* By query params (limited to the top n, and grouping glcid and fbclid)

... and for each one give the v2 percentage (which might need another chart)

## Distribution Over Time

In [262]:
# Group by hour and plot with time axis
events.groupby([events.sent_at_timestamp.hour, events.is_test]).count()

AttributeError: 'Series' object has no attribute 'hour'

# Scroll Depth Distributions

By split:
* How far down the page does a user scroll on average (table and plot)
* Time to reach a point down the page (charts) 


# Pageview Per Session

* Where they click (future)
* 

# Exploring by Browser / Device

We've seen issues on Babe, UCBrowser etc.  Need to check if there are issues from those platforms.

The issues we saw likely resulted in scroll depth issues.  I'm also not sure what analytics we're going to get from those platform, so just generally need to poke it.

## Hacky Events By Device Type

In [262]:
events.head()
events[events.event_name=="PageView"].head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-10-12 18:52:28.633,2019-10-12T18:52:28.633Z,page,PageView,None,0,4a298a33-84b7-444e-96d7-37fe20889347,https://www.moneysmart.id/mobil-arteria-dahlan/,https://www.moneysmart.id/mobil-arteria-dahlan...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,7.1.1,Mozilla/5.0 (Linux; Android 7.1.1; CPH1717 Bui...
1,2019-10-12 18:52:29.893,2019-10-12T18:52:29.893Z,page,PageView,None,0,29739869-94bd-4c21-8d89-751ef2b204b6,https://www.moneysmart.id/keluarga-halilintar-...,https://www.moneysmart.id/keluarga-halilintar-...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/11.5.1 (Linux; U; Android 27; e...
2,2019-10-12 18:52:29.160,2019-10-12T18:52:29.160Z,page,PageView,None,0,a954d9ba-b4ee-47d2-b09e-cd0ace077603,https://www.moneysmart.id/tahir-masuk-jajajara...,https://www.moneysmart.id/tahir-masuk-jajajara...,,...,True,False,True,mobile,None,None,None,Android,28,BacaBerita App/12.2.2 (Linux; U; Android 28; e...
5,2019-10-12 18:52:27.833,2019-10-12T18:52:27.833Z,page,PageView,None,0,6e600b3b-a905-4281-b9cb-8a9c595276bf,https://www.moneysmart.id/film-live-action-dis...,https://www.moneysmart.id/film-live-action-dis...,https://www.google.co.id/search?q=film+powerfu...,...,False,False,True,mobile,UCBrowser,12,12.13.0.1207,Android,8.1.0,Mozilla/5.0 (Linux; U; Android 8.1.0; en-US; v...
7,2019-10-12 18:52:30.706,2019-10-12T18:52:30.706Z,page,PageView,None,0,92395e5b-2486-4f9a-bac6-c6b7e9371068,https://www.moneysmart.id/keluarga-halilintar-...,https://www.moneysmart.id/keluarga-halilintar-...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/12.1.4 (Linux; U; Android 27; e...


In [302]:
# Get the events for a given day by device type, across all URLs


""", context.device.type as device_type
        , context.browser.name as browser_name
        , context.browser.major as browser_major_version
        , context.browser.version as browser_version
        
        , context.operating_system.name as operating_system_name
        , context.operating_system.version as operating_system_version
        , context.user_agent as user_agent"""

# cols_to_group_by = ["event_name", "status",  "device_type", "browser_name", "browser_version"]
cols_to_group_by = ["event_name", "status",  "device_type", "browser_name", "user_agent"]
cols_to_group_by  = ["event_name", "status",  "device_type", ]
# combined_raw_df  = events.groupby(cols_to_group_by+["is_test",]).size()

combined_raw_df = pd.DataFrame({"count":events.fillna(0).groupby(cols_to_group_by+["is_test",]).size()}).reset_index()  #fillna(-1) is to stop it removing Nulls 

# combined_raw_df[combined_raw_df.event_name=="PageView"]


In [303]:
# combined_raw_df[combined_raw_df.is_test==True]

In [304]:


# join across both sides of the AB test
test_df =combined_raw_df[combined_raw_df.is_test==True]
test_df = test_df.rename(columns={"count":"test_count"}) #in_place = True
control_df = combined_raw_df[combined_raw_df.is_test==False]
control_df = control_df.rename(columns={"count":"control_count"})

print(test_df.head(20))

#print(control_df.head())



#print(test_df.keys)
test_df.reset_index()
#print(20*"-")
#print(test_df.keys())
#print(test_df.columns)
test_df
control_df.reset_index()
test_df = test_df.set_index(cols_to_group_by)
control_df = control_df.set_index(cols_to_group_by)

combined_df = test_df.join(control_df, rsuffix="_").reset_index()
#combined_df.fillna(0) #get rid of NaN

   event_name            status device_type  is_test  test_count
2    PageView                 0     desktop     True        1049
4    PageView                 0      mobile     True       26023
6    PageView                 0      tablet     True         241
9     Reading  Article Body 100     desktop     True         598
11    Reading  Article Body 100      mobile     True        9975
13    Reading  Article Body 100      tablet     True          94
16    Reading   Article Body 25     desktop     True         816
18    Reading   Article Body 25      mobile     True       12638
20    Reading   Article Body 25      tablet     True         117
23    Reading   Article Body 50     desktop     True         749
25    Reading   Article Body 50      mobile     True       11719
27    Reading   Article Body 50      tablet     True         113
30    Reading   Article Body 75     desktop     True         683
32    Reading   Article Body 75      mobile     True       10949
34    Reading   Article B

In [305]:
combined_df["test_fraction"] = combined_df.apply (lambda row: row.test_count/(row.control_count + row.test_count), axis=1)

In [306]:
# combined_df[combined_df['control_count']>1000][combined_df['browser_name']=="Facebook"].sort_values(["browser_name", "browser_version", "status"]).to_csv("results.csv")
# "test_fraction", 

# combined_df[combined_df['control_count']>100].sort_values(["user_agent", "event_name","status"])


combined_df[combined_df['control_count']>100][combined_df['test_count']>50].sort_values(["test_fraction"])


,event_name,status,device_type,is_test,test_count,is_test_,control_count,test_fraction
13,Reading,Article Body 75,mobile,True,10949,False,119469,0.083953
4,Reading,Article Body 100,mobile,True,9975,False,108195,0.084412
10,Reading,Article Body 50,mobile,True,11719,False,125681,0.085291
7,Reading,Article Body 25,mobile,True,12638,False,132508,0.087071
9,Reading,Article Body 50,desktop,True,749,False,7535,0.090415
6,Reading,Article Body 25,desktop,True,816,False,8204,0.090466
0,PageView,0,desktop,True,1049,False,10546,0.090470
12,Reading,Article Body 75,desktop,True,683,False,6844,0.090740
3,Reading,Article Body 100,desktop,True,598,False,5901,0.092014
5,Reading,Article Body 100,tablet,True,94,False,880,0.096509


# Error Logs


## Scalyr

https://www.scalyr.com/help/api#query

* 500s and then with sense check removing bots
* Where status != 200 or a 30x
* All with links to the query dashboard.

Do for:
* ID v1
* ID v2 front end


## Airbrake?? 

* Overall counts probably only

# Search Console.

* Position, clicks, impressions over time, with country filter set to ID
* Track top organic urls and see if there's a different trend

In [312]:
0.0584*24*30

42.048

# Digging Into What Makes Up a Split

* By URL any significant  discrepancy?
* Is it distorted by bots?

## Pages Performing Worse in v2 Than v1

Background to this is that we are seeing a drop off at the top of the page, and that might be due to specific pages, so we want to know if pages are performing poorly:

* Pageviews ratio
* Average scroll depth

# Query Params

In [ ]:
from urllib.parse import urlparse, parse_qs

def extract_query_params_from_url(url):
    url_components = urlparse(url)  # this is also a good way of getting the canonical.
    query_params_dict = parse_qs(url_components.query)
    return query_params_dict
    
    
def extract_reduced_query_params_from_url(url):
    """
    Excludes AMP, and groups together where theree's a unique value e.g. gclid, or fbclid
    """
    
    query_params = extract_query_params_from_url(url)
    unique_per_user_keys = ["fbclid", "gclid"]
    for key in unique_per_user_keys:
        if key in query_params:
            query_params = {key:["unique-per-use-other-keys-removed",]}  # doing this as facebook has loads of stuff
            #query_params[key]=['unqiue_per_user']
    #TODO: >> actually build this.
    #fbclid
    #gclid
    
    return query_params

def extract_reduced_query_params_from_url_to_sorted_tuple(url):
    """
    This is being done to make it hashable for grouping.
    """
    query_params_dict = extract_reduced_query_params_from_url(url)
    keys = list(query_params_dict.keys())
    keys.sort()
    ret = []
    for key in keys:
        ret.append((key, ",".join(query_params_dict[key])))
    
    return tuple(ret) 

non_amp_pageview_query_params = pageview_events[pageview_events.is_amp==False].apply(lambda row: extract_reduced_query_params_from_url_to_sorted_tuple(row["page_url"]), axis=1)
non_amp_pageview_query_params.head()

In [ ]:
non_amp_pageview_query_params[0]

In [ ]:
pqdf = pd.DataFrame()
pqdf["query_params"]=non_amp_pageview_query_params
pqdf["sent_at_timestamp"] = pageview_events[pageview_events.is_amp==False]["sent_at_timestamp"]

In [ ]:
pqdf.head()

In [ ]:
pqdf.dtypes

In [ ]:
query_params_by_date = pqdf.groupby([pqdf.sent_at_timestamp.dt.floor('d'), pqdf.query_params]).count()

In [ ]:
query_params_by_date #.sort_values(query_params_by_date.sent_at_timestamp)